# 11.2 The XML Data Format

**XML**, which stands for eXtensible Markup Language, is another way to represent hierarchical data. The basic building block of XML is the **tag**, denoted by angle brackets `<>`.

For example, a data set of movies might be represented using XML as follows:

```
<movies>
  <movie id="1" title="The Godfather">
    <director id="50" name="Coppola, Francis Ford">
    </director>
    <releasedate>1972-03-24</releasedate>
    <character id="100" name="Vito Corleone">
      <actor id="200" name="Brando, Marlon">
      </actor>
    </character>
    <character id="101" name="Michael Corleone">
      <actor id="201" name="Pacino, Al">
      </actor>
    </character>
    ...
  </movie>
  <movie id="2" title="The Godfather: Part II">
    <director id="50" name="Coppola, Francis Ford">
    </director>
    <releasedate>1974-10-20</releasedate>
    <character id="101" name="Michael Corleone">
      <actor id="201" name="Pacino, Al">
      </actor>
    </character>
    <character id="100" name="Vito Corleone">
      <actor id="250" name="De Niro, Robert">
      </actor>
    </character>
    ...
  </movie>
  ...
</movies>
```

Note the following features of XML:

- Every tag `<a>` has a corresponding closing tag `</a>`. You can always recognize a closing tag by the forward slash `/`.
- Additional tags and/or strings can be nested between the opening and closing tags. In the example above, `<actor>` is nested between `<character>` and `</character>`, and `<character>` is nested between `<movie>` and `</movie>`. The nesting is used to represent hierarchy.
- Indentation is used to make the code more readable (to make it easier to see the nesting structure). But it is optional.
- Attributes can be associated with each tag, like `id=` and `name=` with the `<character>` tag and `id=` and `title=` with the `<movie>` tag.

Each tag represents a variable in the data set. Unlike JSON, which uses lists to represent repeated fields, XML represents repeated fields by simply repeating tags where necessary. In the example above, there are multiple instances of `<movie>` within `<movies>` and multiple instances of `<character>` within `<movie>`, so `movie` and `character` are both repeated fields. (In fact, `director` is also a repeated field, but it is impossible to tell from the code above, since the movies shown above only have one director.)

You will learn XML by working with a data set of programs by the New York Philharmonic (one of the leading orchestras in the world). First, let's take a peek at the data.

In [1]:
import requests
response = requests.get("https://github.com/nyphilarchive/PerformanceHistory/blob/master/Programs/xml/1963-64_TO_1973-74.xml?raw=true")
response.text[:1000]

'<?xml version="1.0" encoding="utf-8"?>\n\n<programs>\n    <program>\n        <id>94a27122-4552-43bc-b89a-7e95ca27900f-0.1</id>\n        <programID>6407</programID>\n        <orchestra>New York Philharmonic</orchestra>\n        <season>1963-64</season>\n        <concertInfo>\n            <eventType>Tour</eventType>\n            <Location>Hollywood, CA</Location>\n            <Venue>Hollywood Bowl</Venue>\n            <Date>1963-09-01T04:00:00Z</Date>\n            <Time>8:30PM</Time>\n        </concertInfo>\n        <worksInfo>\n            <work ID="51807*">\n                <composerName>Brahms,  Johannes</composerName>\n                <workTitle>ACADEMIC FESTIVAL OVERTURE, OP.80</workTitle>\n                <conductorName>Bernstein, Leonard</conductorName>\n            </work>\n            <work ID="51884*">\n                <composerName>Brahms,  Johannes</composerName>\n                <workTitle>SYMPHONY NO. 4 IN E MINOR, OP. 98</workTitle>\n                <conductorName>Bernste

Notice that this XML file is nearly twice as large as the JSON file. Although XML is more readable than JSON, it is a more expensive way to store hierarchical data, primarily because of the cost of storing both the opening and closing tags.

There are several libraries in Python for working with XML, including BeautifulSoup (which we will use in the next section to parse HTML), ElementTree, and `lxml`. We will use `lxml` to work with XML data because it is fastest for large data sets, provided that the data is well-formed. The `lxml` library provides a convenient API that replicates all of the functionality of ElementTree, plus implements a few additional features that are useful for data analysis.

In [2]:
from lxml import etree

First, let's read in the data. Since we have the XML as a string in memory, we use the `etree.fromstring()` to load the data. Note that `.fromstring()` returns the root node.

In [3]:
string = response.text.encode()
etree.fromstring(string)

<Element programs at 0x102ee1c30>

Each direct descendant, or **child**, of `<programs>` is a program. To find the direct descendants of a tag, we call the `.getchildren()` method.

In [4]:
programs = etree.fromstring(string)
print(len(programs.getchildren()))
programs.getchildren()[:10]

968


[<Element program at 0x10444d5f0>,
 <Element program at 0x104439050>,
 <Element program at 0x104437dc0>,
 <Element program at 0x104437d70>,
 <Element program at 0x104437d20>,
 <Element program at 0x104437cd0>,
 <Element program at 0x104437c80>,
 <Element program at 0x104437c30>,
 <Element program at 0x104437be0>,
 <Element program at 0x104437820>]

Let's print out the first of these programs. There are two ways to get the first program.

In [5]:
# METHOD 1: Get it from the list above.
program = programs.getchildren()[0]

# METHOD 2: Use .find() to find the first instance of a tag.
program = programs.find("program")
program

<Element program at 0x10444d5f0>

Now let's see how the data is represented by printing out the XML of this program. To do this, we use the `etree.tostring()` function.

In [6]:
print(etree.tostring(program, encoding="unicode"))

<program>
        <id>94a27122-4552-43bc-b89a-7e95ca27900f-0.1</id>
        <programID>6407</programID>
        <orchestra>New York Philharmonic</orchestra>
        <season>1963-64</season>
        <concertInfo>
            <eventType>Tour</eventType>
            <Location>Hollywood, CA</Location>
            <Venue>Hollywood Bowl</Venue>
            <Date>1963-09-01T04:00:00Z</Date>
            <Time>8:30PM</Time>
        </concertInfo>
        <worksInfo>
            <work ID="51807*">
                <composerName>Brahms,  Johannes</composerName>
                <workTitle>ACADEMIC FESTIVAL OVERTURE, OP.80</workTitle>
                <conductorName>Bernstein, Leonard</conductorName>
            </work>
            <work ID="51884*">
                <composerName>Brahms,  Johannes</composerName>
                <workTitle>SYMPHONY NO. 4 IN E MINOR, OP. 98</workTitle>
                <conductorName>Bernstein, Leonard</conductorName>
            </work>
            <work ID="0*">
     

Hopefully, the structure of this data is self-explanatory. "Work", "concertInfo", and "soloist" are repeated fields inside "program". One difference between JSON and XML is that "work" is not directly nested within "program"; the "work" tags are all nested inside an additional "worksInfo" tag.

Now suppose that we want to flatten the data at the level of soloists. To get all of the soloists, we can use the `.findall()` method. Let's first try the obvious solution, which does not work:

In [7]:
programs.findall("soloist")

[]

Why did `lxml` fail to find any `<soloist>` tags? That's because `.findall()` only searches among the direct descendants of a tag. We called `.findall()` on the `<programs>` tag, but all of its descendants are `<program>` tags.

To specify that `lxml` should look for `<soloist>` tags among all descendants, not just direct ones, we use the `.xpath()` command, which allows us to specify an XPath expression. [XPath](https://www.w3schools.com/xml/xpath_syntax.asp) is a language used to select nodes from XML documents. The XPath expression to select all descendants named `<soloist>` of the current tag is `".//soloist"`. We pass this expression to the `.xpath()` method.

In [8]:
soloists = programs.xpath(".//soloist")
len(soloists)

3493

Now, to flatten the data at the level of soloists, we just need to turn `soloists` into a `DataFrame` with as many rows. But what if we want to include information from parent levels, like the composer of the work the soloist played? There are two ways.

### Method 1

Since `<composerName>` is a descendant of `<work>`, one way is to navigate up to the level of `<work>` by calling `.getparent()` repeatedly and then find `<composerName>` among its descendants:

In [9]:
soloist = soloists[0]

# The first .getparent() returns the <soloists> tag.
# The second .getparent() returns the <work> tag.
# You have to figure this out by inspecting the XML.
work = soloist.getparent().getparent()
work.xpath(".//composerName")

[<Element composerName at 0x1043c3c80>]

This is a list with one tag, so we extract that tag and the text inside it.

In [10]:
work.xpath(".//composerName")[0].text

'Brahms,  Johannes'

### Method 2

As the number of levels of nesting increases, it quickly becomes impractical to call `.getparent()` repeatedly. We want to be able to jump directly to the right ancestor. The easiest way to do this is to use the XPath expression for an ancestor. To search for all ancestors named "work", we can use the XPath expression `"ancestor::work"`.

In [11]:
soloist.xpath("ancestor::work")

[<Element work at 0x1043c3c30>]

Now, we can extract this single work tag and find its descendants named `<composerName>`. Or better yet, we can combine this step with the above step into a single XPath expression.

In [12]:
soloist.xpath("ancestor::work//composerName")[0].text

'Brahms,  Johannes'

Now let's put it all together. We will flatten the data to get a `DataFrame` with one soloist per row. We will keep track of the soloist's name, instrument, and role---as well as the composer of the work they performed. Unfortunately, it is much more manual to do this with XML than with JSON. There is no XML equivalent of the `json_normalize` function that will automatically produce a `DataFrame`, so we have to construct the `DataFrame` ourselves.

In [13]:
import pandas as pd

rows = []

soloists = programs.xpath(".//soloist")
for soloist in soloists:
    row = {}
    row["soloistName"] = soloist.find("soloistName").text
    row["soloistInstrument"] = soloist.find("soloistInstrument").text
    row["soloistRoles"] = soloist.find("soloistRoles").text
    row["composerName"] = soloist.xpath("ancestor::work//composerName")[0].text
    rows.append(row)
    
df_soloists = pd.DataFrame(rows)
df_soloists

,soloistName,soloistInstrument,soloistRoles,composerName
0,"Heifetz, Jascha",Violin,S,"Brahms, Johannes"
1,"Piatigorsky, Gregor",Cello,S,"Brahms, Johannes"
2,"Watts, Andre (André)",Piano,S,"Liszt, Franz"
3,"Curtin, Phyllis",Soprano,S,"Strauss, Richard"
4,"Raskin, Judith",Soprano,S,"Strauss, Richard"
...,...,...,...,...
3488,"Cerminaro, John",French Horn,A,"Mahler, Gustav"
3489,"Bernstein, Leonard",Piano,S,"Mozart, Wolfgang Amadeus"
3490,"Cerminaro, John",French Horn,A,"Mahler, Gustav"
3491,"Bernstein, Leonard",Piano,S,"Mozart, Wolfgang Amadeus"


Now, this is a `DataFrame` that we can analyze easily. For example, here is how many times Benny Goodman programmed a work by Weber with the NY Phil:

In [14]:
df_soloists[df_soloists["soloistName"] == "Goodman, Benny"].composerName.value_counts()

Weber,  Carl  Maria Von       2
Copland,  Aaron               1
Mozart,  Wolfgang  Amadeus    1
Name: composerName, dtype: int64

# Exercises

Exercises 1 and 2 deal with the New York Philharmonic data set from above.

1\. What is the most frequent start time for New York Philharmonic concerts?

2\. How many distinct works did the New York Philharmonic perform in the 1970-71 season?

In Exercise 3, you will work with [data from the Meteorologisk Institutt of Norway](https://api.met.no/weatherapi/documentation), which returns weather data in XML form.

If you run into unexpected errors, issue the same HTTP request from your browser to make sure that the data is in the format you expect.

3\. Make a graph showing the forecasted temperature for the upcoming week in Oslo, Norway. Make a graphic that displays this information.

(Hint: You may want to look up the latitude and longitude of Oslo.)